CREATING A SPARK SESSION

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("Log_Analysis").getOrCreate()

CODE TO GET THE DATA FROM AZURE STORAGE ACCOUNT INTO DATABRICKS

In [0]:
storage_account_name= "my2demo1057"
storage_account_key= "j051LupqroJJEKyyU1dwIQjf8L0B6oJQ7xaPcxGpZv/xFnaokvWLhb0fBZ/lXzxiOZCwqLum/99L+AStFTlSDg=="
container= "containerproject"

In [0]:
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net",storage_account_key)

In [0]:
dbutils.fs.ls(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/")

[FileInfo(path='wasbs://containerproject@my2demo1057.blob.core.windows.net/log_data.csv', name='log_data.csv', size=6743, modificationTime=1709188754000)]

LOADING DATA INTO DATAFRAME

In [0]:
df=spark.read.csv(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/",header=True, inferSchema=True)

In [0]:
df.show(10)

+--------------------+---------------+--------------+-----------+----------+
|           Timestamp|     IP Address|    User Agent|HTTP Status|Bytes Sent|
+--------------------+---------------+--------------+-----------+----------+
|2023-04-16 05:13:...|  192.168.87.28|UserAgent-5989|        200|      3738|
|2023-08-29 02:58:...|  192.168.97.78|UserAgent-5915|        200|      6050|
|2023-12-16 17:36:...|   192.168.1.40|UserAgent-2408|        200|      3640|
|2024-02-08 21:38:...| 192.168.63.146|UserAgent-7203|        200|      8178|
|2023-07-15 05:30:...|192.168.139.169|UserAgent-5160|        500|      4085|
|2023-10-13 14:08:...|  192.168.59.34|UserAgent-5856|        200|      1060|
|2024-02-14 11:35:...| 192.168.160.40|UserAgent-3677|        200|      1224|
|2023-03-15 17:12:...|192.168.142.236|UserAgent-7323|        200|       877|
|2023-06-09 20:46:...| 192.168.151.51|UserAgent-1747|        404|      7617|
|2023-04-13 03:56:...| 192.168.244.63|UserAgent-4898|        500|      6967|

INSTEAD OF EXPLICITLY SPECIFYING THE FULL PATH EVERY TIME TO ACCESS THE DATA, WE CAN USE MOUNTING.

In [0]:
dbutils.fs.mount(
    source=f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/",
    mount_point="/mnt/storage",
    extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
)

True

In [0]:
dbutils.fs.mounts()

[MountInfo(mountPoint='/mnt/storage', source='wasbs://containerproject@my2demo1057.blob.core.windows.net/', encryptionType=''),
 MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/Volume', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/volumes', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType=''),
 MountInfo(mountPoint='/volume', source='DbfsReserved', encryptionType='')]

In [0]:
df = spark.read.csv("/mnt/storage/log_data.csv", header=True, inferSchema=True)

In [0]:
df.show(10)

+--------------------+---------------+--------------+-----------+----------+
|           Timestamp|     IP Address|    User Agent|HTTP Status|Bytes Sent|
+--------------------+---------------+--------------+-----------+----------+
|2023-04-16 05:13:...|  192.168.87.28|UserAgent-5989|        200|      3738|
|2023-08-29 02:58:...|  192.168.97.78|UserAgent-5915|        200|      6050|
|2023-12-16 17:36:...|   192.168.1.40|UserAgent-2408|        200|      3640|
|2024-02-08 21:38:...| 192.168.63.146|UserAgent-7203|        200|      8178|
|2023-07-15 05:30:...|192.168.139.169|UserAgent-5160|        500|      4085|
|2023-10-13 14:08:...|  192.168.59.34|UserAgent-5856|        200|      1060|
|2024-02-14 11:35:...| 192.168.160.40|UserAgent-3677|        200|      1224|
|2023-03-15 17:12:...|192.168.142.236|UserAgent-7323|        200|       877|
|2023-06-09 20:46:...| 192.168.151.51|UserAgent-1747|        404|      7617|
|2023-04-13 03:56:...| 192.168.244.63|UserAgent-4898|        500|      6967|

In [0]:
df.printSchema()

root
 |-- Timestamp: timestamp (nullable = true)
 |-- IP Address: string (nullable = true)
 |-- User Agent: string (nullable = true)
 |-- HTTP Status: integer (nullable = true)
 |-- Bytes Sent: integer (nullable = true)



In [0]:
display(df)

Timestamp,IP Address,User Agent,HTTP Status,Bytes Sent
2023-04-16T05:13:53.659803Z,192.168.87.28,UserAgent-5989,200,3738
2023-08-29T02:58:07.659803Z,192.168.97.78,UserAgent-5915,200,6050
2023-12-16T17:36:16.659803Z,192.168.1.40,UserAgent-2408,200,3640
2024-02-08T21:38:23.659803Z,192.168.63.146,UserAgent-7203,200,8178
2023-07-15T05:30:23.659803Z,192.168.139.169,UserAgent-5160,500,4085
2023-10-13T14:08:09.659803Z,192.168.59.34,UserAgent-5856,200,1060
2024-02-14T11:35:54.659803Z,192.168.160.40,UserAgent-3677,200,1224
2023-03-15T17:12:12.659803Z,192.168.142.236,UserAgent-7323,200,877
2023-06-09T20:46:27.659803Z,192.168.151.51,UserAgent-1747,404,7617
2023-04-13T03:56:16.659803Z,192.168.244.63,UserAgent-4898,500,6967


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

HANDLING MISSING VALUES

In [0]:
df = df.na.drop()

LETS COMPUTE BASIC STATISTICS FOR NUMERICAL COLUMNS

In [0]:
df.describe().show()

+-------+-------------+--------------+------------------+------------------+
|summary|   IP Address|    User Agent|       HTTP Status|        Bytes Sent|
+-------+-------------+--------------+------------------+------------------+
|  count|          100|           100|               100|               100|
|   mean|         NULL|          NULL|            350.48|           4856.91|
| stddev|         NULL|          NULL|124.12802774847937|2612.2662791205453|
|    min| 192.168.1.40|UserAgent-1007|               200|               384|
|    max|192.168.99.51|UserAgent-9872|               500|              9934|
+-------+-------------+--------------+------------------+------------------+



FINDING ERROR RATE

In [0]:
total_records = df.count()

In [0]:
from pyspark.sql.functions import col
error_records = df.filter((col('HTTP Status') == 404) | (col('HTTP Status') == 500)).count()

In [0]:
error_rate = (error_records / total_records) * 100
error_rate

62.0

ANALYZING TRENDS
(WE CAN ANALYZE TRENDS BY GROUPING DATA BY A CERTAIN TIME INTERVAL)

In [0]:
trends = df.groupBy(df.Timestamp.cast("date").alias("Date"), "HTTP Status").count().orderBy("Date")

TREND OF HTTP STATUS OVER TIME

In [0]:
trends.show(10)

+----------+-----------+-----+
|      Date|HTTP Status|count|
+----------+-----------+-----+
|2023-03-08|        404|    1|
|2023-03-15|        200|    1|
|2023-03-23|        404|    1|
|2023-03-28|        404|    1|
|2023-04-07|        200|    1|
|2023-04-13|        500|    1|
|2023-04-15|        200|    1|
|2023-04-16|        200|    1|
|2023-04-22|        200|    1|
|2023-05-06|        404|    1|
+----------+-----------+-----+
only showing top 10 rows



TREND OF IP ADDRESSES OVER TIME

In [0]:
IP_trends = df.groupBy(df.Timestamp.cast("date").alias("Date"), "IP Address").count().orderBy("Date")

TRENDS OF REQUESTS FROM IP ADDRESSES OVER TIME

In [0]:
IP_trends.show(10)

+----------+---------------+-----+
|      Date|     IP Address|count|
+----------+---------------+-----+
|2023-03-08| 192.168.93.156|    1|
|2023-03-15|192.168.142.236|    1|
|2023-03-23|  192.168.68.41|    1|
|2023-03-28|192.168.130.149|    1|
|2023-04-07|  192.168.37.37|    1|
|2023-04-13| 192.168.244.63|    1|
|2023-04-15| 192.168.45.223|    1|
|2023-04-16|  192.168.87.28|    1|
|2023-04-22| 192.168.207.20|    1|
|2023-05-06|   192.168.3.69|    1|
+----------+---------------+-----+
only showing top 10 rows

